In [113]:
# nuclio: ignore
import nuclio

In [114]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/mlrun:0.5.1"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/mlrun:0.5.1'


In [115]:
# nuclio: start-code

In [116]:
import os
import json
import pandas as pd
from mlrun.datastore import DataItem

In [117]:
def save_property(context,source_file='',target_file=''):
    #source_file = context.get_param('source_file')
    #target_file = context.get_param('target_file')
    print(source_file)
    all_properties_df = pd.read_csv(source_file)
    condo_properties_df = all_properties_df[all_properties_df["type"]=="Condo"]
    
    # Record a result 
    context.log_result('properties_available',condo_properties_df.shape[0])
    print("new version: V2")
    #Record a Dataset
    context.log_dataset('condo_properties',df=condo_properties_df,format="csv")
    return

In [118]:
def filter_properties(context,
                      source_data: DataItem,
                      target_dataset='',
                      city=''):
        
    df = source_data.as_df()
    
    city_df = df[df["city"]==city]
    
    context.log_dataset(target_dataset,df=city_df,format="csv")

In [119]:
# nuclio: end-code

# Save as market function

In [120]:
from mlrun import code_to_function

# create job function object from notebook code
fn = code_to_function("properties-management")
# add metadata (for templates and reuse)
fn.spec.default_handler = "save_property"
fn.spec.description = "Filter properties"
fn.metadata.categories = ["bi"]
fn.metadata.labels = {"author": "mdl"}
fn.metadata.tag = "v2"
fn.export("function.yaml")

> 2020-09-29 18:58:50,901 [info] function spec saved to path: function.yaml
